##### README
- the goal of this notebook is to call a tool within LangChain

In [4]:
import dotenv
dotenv.load_dotenv(override=True)

True

In [5]:
import os
import sys

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

if not os.environ.get("OPENAI_API_KEY"):
    print("Please set your OPENAI_API_KEY environment variable.")
    # sys.exit(1) # Uncomment to enforce exit if no key found

C:\Users\Danny\Desktop\tmp\venvs\langgraph\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [6]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0)

In [7]:
# 2. Instantiate the Chat Model
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ee69c2ef48', 'id': 'chatcmpl-Cns6I8dIyJp5yyEqIjRAqsZvV9bP0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2ded-1310-76b1-add6-1c7b1c703716-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
# 1. Define a simple Python function and decorate it as a LangChain Tool
# The docstring is crucial, as it tells the LLM what the tool does.
@tool
def get_current_weather(city: str) -> str:
    """
    Returns the current weather in a given city.
    For this example, it only returns a hardcoded string.
    """
    if "boston" in city.lower():
        return "It is 15°C and sunny in Boston."
    elif "richmond hill" in city.lower():
        # Personalization: Richmond Hill is where your son's daycare is located.
        return "It is 5°C and snowing heavily in Richmond Hill."
    else:
        return f"Weather data for {city} is not available in this toy tool."

In [9]:
# 3. Bind the Tool to the Model
# This is the core step. The model's configuration is updated to know about the tool.
llm_with_tool = llm.bind_tools([get_current_weather])

In [10]:
# 4. Define the Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the available tool to answer questions about the weather."),
    ("user", "{input}"),
])

In [11]:
# 5. Create the Chain
chain = prompt | llm_with_tool

In [12]:
general_query = "what is the temperature in Boston?"
print(f"User Query: {general_query}\n")

response = chain.invoke({"input": general_query})

User Query: what is the temperature in Boston?



In [13]:
print(response.content)
print(response.tool_calls)


[{'name': 'get_current_weather', 'args': {'city': 'Boston'}, 'id': 'call_42sMPMSLqGfN1M5ziLGTCEyW', 'type': 'tool_call'}]


##### Invoke LLM directly

In [14]:
r = llm_with_tool.invoke([
	("system", "You're a helpful assistant"), 
	("human", "what's 5 raised to the 2.743"),
])

In [15]:
r.content

'To calculate \\( 5^{2.743} \\), you can use the formula for exponentiation. \n\nCalculating \\( 5^{2.743} \\) gives approximately \\( 36.206 \\).'

In [16]:
r.tool_calls

[]

In [17]:
r = llm_with_tool.invoke([
	("system", "You are a helpful assistant. Use the available tool to answer questions about the weather."), 
	("human", "what's is the weather in Boston"),
])

In [48]:
r.content

''

In [49]:
r.tool_calls

[{'name': 'get_current_weather',
  'args': {'city': 'Boston'},
  'id': 'call_yDUuXNQYVH4sev0KBqYA9Lb1',
  'type': 'tool_call'}]

In [ ]:
# note that the tool is not actually being executed ... 